## 📂 Cell 1: Mount Google Drive

In [ ]:
from pathlib import Path

# ⚠️ KAGGLE SETUP - Choose your option:

# OPTION A: Dataset uploaded to Kaggle (RECOMMENDED)
# After uploading to Kaggle Datasets, use this:
# DATA_ROOT = Path('/kaggle/input/nsclc-multiorgan-segmentation')

# OPTION B: Dataset on external source (slower)
print("""
╔═══════════════════════════════════════════════════════════════╗
║  📌 SETUP INSTRUCTIONS FOR KAGGLE                             ║
╠═══════════════════════════════════════════════════════════════╣
║  1. Upload kaggle_dataset.zip to Kaggle Datasets              ║
║  2. In this notebook: Click "+ Add Data" (top right)          ║
║  3. Search for your dataset name                              ║
║  4. Add it to this notebook                                    ║
║  5. Update DATA_ROOT below with correct path                  ║
╚═══════════════════════════════════════════════════════════════╝
""")

# TODO: Update this path after adding your dataset
DATA_ROOT = Path('/kaggle/input/nsclc-multiorgan-segmentation/nsclc_multiorgan_segmentation')

print(f"Dataset path: {DATA_ROOT}")
print("⚠️ Make sure to update DATA_ROOT with your actual dataset path!")

## 🔧 Cell 2: Setup Paths & Check Dataset

In [ ]:
import sys
from pathlib import Path

# Update this after adding your dataset to Kaggle
DATA_ROOT = Path('/kaggle/input/YOUR-DATASET-NAME-HERE/nsclc_multiorgan_segmentation')

# Check if path exists
if not DATA_ROOT.exists():
    print("❌ Dataset not found!")
    print(f"❌ Path: {DATA_ROOT}")
    print("\n📝 To fix:")
    print("  1. Click '+ Add Data' (top right)")
    print("  2. Search for your uploaded dataset")
    print("  3. Add it to this notebook")
    print("  4. Update DATA_ROOT above with correct path")
else:
    print("✅ Dataset found!")
    print(f"📂 Path: {DATA_ROOT}")
    
    # Check structure
    ct_files = list((DATA_ROOT / 'normalized_ct').glob('*.nii.gz'))
    mask_files = list((DATA_ROOT / 'normalized_masks').glob('*.nii.gz'))
    code_files = list((DATA_ROOT / 'code').glob('*.py'))
    
    print(f"✅ CT scans: {len(ct_files)} files")
    print(f"✅ Masks: {len(mask_files)} files")
    print(f"✅ Code files: {code_files}")
    
    # Add code to path
    sys.path.append(str(DATA_ROOT / 'code'))

## 📦 Cell 3: Install Dependencies & Imports

In [ ]:
# Install SimpleITK if needed
!pip install SimpleITK -q

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import SimpleITK as sitk

# Import custom modules
from dataset_multi_organ import MultiOrganDataset
from unet_multi_organ import UNetMultiOrgan

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## ⚙️ Cell 4: Configuration

In [ ]:
CONFIG = {
    'batch_size': 8,
    'num_epochs': 50,
    'learning_rate': 1e-4,
    'num_workers': 2,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'patience': 10,
    'in_channels': 1,
    'out_channels': 8,
}

print("Configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

## 📊 Cell 5: Load Datasets

In [ ]:
print("Loading datasets...")

train_dataset = MultiOrganDataset(
    data_root=str(DATA_ROOT),
    split='train',
    slice_wise=True
)

val_dataset = MultiOrganDataset(
    data_root=str(DATA_ROOT),
    split='val',
    slice_wise=True
)

train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

print(f"✅ Train: {len(train_dataset)} slices")
print(f"✅ Val: {len(val_dataset)} slices")

## 🧠 Cell 6: Initialize Model

In [ ]:
model = UNetMultiOrgan(
    in_channels=1,
    out_channels=8,
    bilinear=False
).to(CONFIG['device'])

num_params = sum(p.numel() for p in model.parameters())
print(f"✅ Model: {num_params:,} parameters")
print(f"✅ Model size: {num_params * 4 / 1024**2:.2f} MB")

## 🎯 Cell 7: Loss Functions & Optimizer

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super().__init__()
        self.smooth = smooth
    
    def forward(self, pred, target):
        pred = torch.softmax(pred, dim=1)
        target_one_hot = torch.nn.functional.one_hot(target, 8).permute(0, 3, 1, 2).float()
        
        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1.0 - dice.mean()

class CombinedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.ce = nn.CrossEntropyLoss()
        self.dice = DiceLoss()
    
    def forward(self, pred, target):
        return 0.5 * self.ce(pred, target) + 0.5 * self.dice(pred, target)

criterion = CombinedLoss()
optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'])

print("✅ Loss: CombinedLoss (CE + Dice)")
print("✅ Optimizer: Adam")

## 🔄 Cell 8: Training Functions

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    
    pbar = tqdm(loader, desc='Training')
    for batch in pbar:
        images = batch['image'].to(device)
        masks = batch['mask'].to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / len(loader)

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for batch in pbar:
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / len(loader)

print("✅ Training functions ready")

## 🚀 Cell 9: Main Training Loop

In [ ]:
print("="*80)
print("🚀 STARTING TRAINING")
print("="*80)

history = {'train_loss': [], 'val_loss': []}
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(CONFIG['num_epochs']):
    print(f"\n📊 Epoch {epoch+1}/{CONFIG['num_epochs']}")
    
    train_loss = train_epoch(model, train_loader, criterion, optimizer, CONFIG['device'])
    val_loss = validate(model, val_loader, criterion, CONFIG['device'])
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), '/content/best_model.pth')
        print("✅ Saved best model")
    else:
        patience_counter += 1
        if patience_counter >= CONFIG['patience']:
            print(f"⚠️ Early stopping at epoch {epoch+1}")
            break

print("="*80)
print("✅ TRAINING COMPLETED")
print(f"Best Val Loss: {best_val_loss:.4f}")
print("="*80)

## 📊 Cell 10: Plot Training Results

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss', marker='o')
plt.plot(history['val_loss'], label='Val Loss', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('/content/training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"📊 Best Val Loss: {min(history['val_loss']):.4f}")
print(f"📊 Final Train Loss: {history['train_loss'][-1]:.4f}")

## 🎨 Cell 11: Visualize Predictions

In [ ]:
# Charger best model
model.load_state_dict(torch.load('/content/best_model.pth'))
model.eval()

# Prendre 4 samples du validation set
val_samples = [val_dataset[i] for i in range(0, len(val_dataset), len(val_dataset)//4)][:4]

fig, axes = plt.subplots(4, 3, figsize=(12, 16))

with torch.no_grad():
    for idx, sample in enumerate(val_samples):
        image = sample['image'].unsqueeze(0).to(CONFIG['device'])
        mask_true = sample['mask'].numpy()
        
        # Prédiction
        output = model(image)
        mask_pred = torch.argmax(output, dim=1).cpu().numpy()[0]
        
        # Plot
        axes[idx, 0].imshow(image.cpu().squeeze(), cmap='gray')
        axes[idx, 0].set_title('CT Scan')
        axes[idx, 0].axis('off')
        
        axes[idx, 1].imshow(mask_true, cmap='tab10', vmin=0, vmax=7)
        axes[idx, 1].set_title('Ground Truth')
        axes[idx, 1].axis('off')
        
        axes[idx, 2].imshow(mask_pred, cmap='tab10', vmin=0, vmax=7)
        axes[idx, 2].set_title('Prediction')
        axes[idx, 2].axis('off')

plt.tight_layout()
plt.savefig('/content/predictions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualizations saved")

## 💾 Cell 12: Save Results to Google Drive

In [ ]:
# Copier les résultats vers Drive
!cp /content/best_model.pth "/content/drive/MyDrive/NSCLC_MultiOrgan_Dataset/"
!cp /content/training_curves.png "/content/drive/MyDrive/NSCLC_MultiOrgan_Dataset/"
!cp /content/predictions.png "/content/drive/MyDrive/NSCLC_MultiOrgan_Dataset/"

print("✅ Results saved to Google Drive!")
print("📂 Location: MyDrive/NSCLC_MultiOrgan_Dataset/")